# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Hard: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 

# Download dataset

In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
# From Google drive
# !gdown --id '1T0RPnu-Sg5eIPwQPfYysipfcz81MnsYe' --output Dataset.zip
# !unzip Dataset.zip

# From Dropbox
# If Dropbox is not work. Please use google drive.
# !wget https://www.dropbox.com/s/vw324newiku0sz0/Dataset.tar.gz.aa?dl=0
# !wget https://www.dropbox.com/s/z840g69e7lnkayo/Dataset.tar.gz.ab?dl=0
# !wget https://www.dropbox.com/s/hl081e1ggonio81/Dataset.tar.gz.ac?dl=0
# !wget https://www.dropbox.com/s/fh3zd8ow668c4th/Dataset.tar.gz.ad?dl=0
# !wget https://www.dropbox.com/s/ydzygoy2pv6gw9d/Dataset.tar.gz.ae?dl=0
# !cat Dataset.tar.gz.* | tar zxvf -

# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [3]:
import os
import json
import torch
import random
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=256):
    self.data_dir = data_dir
    self.segment_len = segment_len

    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]

    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]

    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num


## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


In [5]:
class Log:
    def __init__(self, log_path='log', clear=True):
        self.log_path = log_path
        if clear: self.clean_file()
        self.log = open(log_path, 'a')
    
    def clean_file(self):
        if self.log_path in os.listdir():
            os.remove(self.log_path)
            print('remove log')
        
    def flush(self):
        self.log.flush()
    
    def write(self, log_str, is_print=True):
        self.log.write(log_str + "\n")
        if is_print: print(log_str)
    
    def write_epoch(self, total, cur, sections, is_write=True, is_print=True):
        log_str = f"[ {cur + 1:03d}/{total:03d} ]"
        for n, s in sections:
            log_str += f" {self.section(n, s)}"
        if is_write: self.write(log_str, is_print=is_print)
        else: return log_str
    
    def section(self, dest, dict):
        log_str = f"[ {dest:<5} |"
        flg = False
        for n, v in dict.items():
            if flg:
                log_str += ","
            log_str += f" {n} = {v:.5f}"
            flg = True
        log_str += " ]"
        return log_str
    

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [6]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(52728)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [8]:
# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=20000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        print(pe.shape)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

In [9]:
class AttentionPooling(nn.Module):
    def __init__(self, d_model, n_hidden, n_head, dropout=.5):
        super().__init__()
        self.l1 = nn.Linear(d_model, n_hidden, bias=True)
        self.l2 = nn.Linear(n_hidden, n_head, bias=True)
        self.dropout = nn.Dropout(dropout)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        out = self.dropout(x)
        out = self.act(self.l1(out))
        out = self.dropout(out)
        out = self.softmax(self.l2(out))
        return out
        

### Conformer from https://github.com/lucidrains/conformer

In [10]:
from conformer import ConformerBlock
class Classifier(nn.Module):
  def __init__(self, d_model=128, n_spks=600, dropout=0.5):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.d_model = d_model
    self.n_head = 8

    self.prenet = nn.Linear(40, d_model)

    # self.pe = PositionalEncoding(d_model=d_model, max_len=10000)
    # self.encoder_layer = nn.TransformerEncoderLayer(
    #   d_model=d_model, dim_feedforward=256, nhead=4, dropout=dropout
    # )
    self.encoder = [
      ConformerBlock(
          dim = d_model,
          dim_head = d_model//4,
          heads = 8,
          ff_mult = 2,
          conv_expansion_factor = 2,
          conv_kernel_size = 17,
          attn_dropout = dropout,
          ff_dropout = dropout,
          conv_dropout = dropout
      ) for i in range(2)]
    self.encoder = nn.ModuleList(self.encoder)

    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4, norm=nn.LayerNorm(d_model))
    # Project the the dimension of features from d_model into speaker nums.
    self.attention_pool = AttentionPooling(self.d_model, self.d_model, self.n_head, dropout=dropout)
    pred_d = d_model*self.n_head
    self.pred_layer = nn.Sequential(
      nn.Dropout(dropout),
      nn.Linear(pred_d, d_model),
      nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

    self.flat = nn.Flatten()

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    # out = self.prenet(mels) * math.sqrt(self.d_model)
    out = self.prenet(mels)

    # out: (length, batch size, d_model)
    # out = out.permute(1, 0, 2)
    # out = self.pe(out)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    # out = self.encoder(out)

    # out: (batch size, length, d_model)
    for m in self.encoder:
      out = m(out)

    # out: (batch size, length, d_model)
    # out = out.transpose(0, 1)
    # mean pooling
    # stats = out.mean(dim=1)

    # attention: (batch, length, d_model) -> (batch, length, head)
    attention = self.attention_pool(out)
    # print(attention.shape)

    # (batch, d_model, head)
    stats = torch.bmm(out.transpose(1, 2), attention)
    stats = self.flat(stats)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [11]:
import math
import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [12]:
import torch

def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [13]:
from tqdm.auto import tqdm
import torch


def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), desc="Valid", leave=False)

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.update(dataloader.batch_size)
  pbar.close()
  model.train()

  return running_accuracy / len(dataloader), running_loss / len(dataloader)


# Main function

In [14]:
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    # "data_dir": "./Dataset",
    "data_dir": "/data/ML2021/hw4/Dataset",
    "save_path": "model.ckpt",
    "batch_size": 64,
    "n_workers": 8,
    "valid_steps": 1000,
    "warmup_steps": 1000,
    "save_steps": 2000,
    "total_steps": int(5e4),
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  log = Log()
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(save_path))

  criterion = nn.CrossEntropyLoss()
  # criterion = AdMSoftmaxLoss(speaker_num, speaker_num).to(device)
  optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=5e-4)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, desc="Train", leave=False)
  n_total = total_steps // valid_steps
  count = 0
  losses = []
  accs = []
  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)
    loss, accuracy = model_fn(batch, model, criterion, device)
    
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()
    losses.append(batch_loss)
    accs.append(batch_accuracy)

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy, valid_loss = valid(valid_loader, model, criterion, device)
      dicts = []
      dicts.append(['Train', 
        {
          'loss': np.mean(losses),
          'acc': np.mean(accs)
        }])
      dicts.append(['Valid', 
        {
          'loss': valid_loss,
          'acc': valid_accuracy
        }])
      
      log.write_epoch(n_total, count, dicts)
      losses.clear()
      accs.clear()
      count += 1

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()
        log.write(f"Best valid acc: {valid_accuracy:.4f}")

      pbar = tqdm(total=valid_steps, desc="Train", leave=False)
      log.flush()

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      log_str = f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})"
      log.write(log_str)
      log.flush()

  pbar.close()


if __name__ == "__main__":
  torch.cuda.empty_cache()
  main(**parse_args())


remove log
[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Valid:   0%|          | 0/6944 [00:00<?, ?it/s]

[ 001/050 ] [ Train | loss = 0.14302, acc = 0.95620 ] [ Valid | loss = 0.88654, acc = 0.84664 ]
Best valid acc: 0.8466


Train:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Valid:   0%|          | 0/6944 [00:00<?, ?it/s]

[ 050/050 ] [ Train | loss = 0.03689, acc = 0.98887 ] [ Valid | loss = 0.49122, acc = 0.92954 ]


Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Step 50000, best model saved. (accuracy=0.9332)


# Inference

## Dataset of inference

In [14]:
torch.cuda.empty_cache()

In [15]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


## Main funcrion of Inference

In [18]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "/data/ML2021/hw4/Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  # model.pe.pe = model.pe.pe.repeat(5, 1, 1)
  # print(model.pe.pe.shape)
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  count = 0
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      # print(mels.shape)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/6000 [00:00<?, ?it/s]

In [19]:
torch.cuda.empty_cache()